In [1]:
import pandas as pd
import requests as r
import json

In [22]:
df = pd.DataFrame()
for i in range(1,471):
    url = f'https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page={i}'
    response = r.get(url)
    data = response.json()
    temp_df = pd.json_normalize(data['results'])[['title','overview','genre_ids']]
    df = pd.concat([df, temp_df], ignore_index=True)

In [23]:
df

,title,overview,genre_ids
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[18, 80]"
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[18, 80]"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[18, 80]"
3,Schindler's List,The true story of how businessman Oskar Schind...,"[18, 36, 10752]"
4,12 Angry Men,The defense and the prosecution have rested an...,[18]
...,...,...,...
9378,Alone in the Dark,Edward Carnby is a private investigator specia...,"[28, 14, 27]"
9379,Disaster Movie,"The filmmaking team behind the hits ""Scary Mov...","[35, 878]"
9380,House of the Dead,"Set on an island off the coast, a techno rave ...","[27, 28, 53]"
9381,Dragonball Evolution,"On his 18th birthday, Goku receives a mystical...","[28, 12, 14, 878, 53]"


In [48]:
df.to_csv('movies.csv',index=False)

In [11]:
# convert in lower case
df['overview'] = df['overview'].str.lower()

In [14]:
# remove puntuations
import string
exclude = string.punctuation
def remove_punt(text):
    return text.translate(str.maketrans('','',exclude))

In [20]:
df['overview'].isna().sum()

1

In [21]:
df.dropna(inplace=True)

In [23]:
df['overview'] = df['overview'].apply(remove_punt)

In [28]:
# speling check
from textblob import TextBlob

def spell_correct(sentence):
    text_blb = TextBlob(sentence)
    return text_blb.correct().string

df['overview'] = df['overview'].apply(spell_correct)

In [ ]:
# remove stopwords
from nltk.corpus import stopwords

def remove_stopwords(sent):
    new_sent = []
    for word in sent.split():
        if word in stopwords.words('english'):
            new_sent.append('')
        else:
            new_sent.append(word)
    x = new_sent[:]
    new_sent.clear()
    return " ".join(x)

In [ ]:
# tokenization
import spacy 
spacy.cli.download("en_core_web_sm")
nlp = spacy.load('en_core_web_sm')